# Packages

In [4]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import json
from glob import glob

# Parameters

In [2]:
IMG_PATH = '../input/original/' # input
MASK_PATH = '../input/mask/' # output
region_attribute = 'GAP'
region_names = ['gap', 'scale']

# Image and mask.json

In [7]:
with open(MASK_PATH + 'mask.json') as f:
    mask = json.load(f)
    
cmap = plt.get_cmap("tab10")
df = pd.DataFrame(columns=['filename', 'labels', 'contours'])
fig, axes = plt.subplots(2, min(len(mask),5), figsize=(20,6))
for i, k in enumerate(mask.keys()):
    f = mask[k]['filename']
    img = cv2.imread(IMG_PATH + f)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_w_mask = img.copy()
    contours = []
    labels = []
    for r in mask[k]['regions'].keys():
        x = mask[k]['regions'][r]['shape_attributes']['all_points_x']
        y = mask[k]['regions'][r]['shape_attributes']['all_points_y']
        label = mask[k]['regions'][r]['region_attributes'][region_attribute]
        contour = np.vstack([x, y]).T
        for j, region_name in enumerate(region_names):
            if label == region_names[j]:
                cv2.drawContours(img_w_mask, [contour], -1, thickness=-1, color=np.array(cmap(j))*255)
        
        contours.append(contour)
        labels.append(label)

    df.loc[i] = [f, labels, contours]
    
    if i<5:
        axes[0][i].imshow(img)
        axes[0][i].set_title(f)
        axes[0][i].axis('off')
        axes[1][i].imshow(img_w_mask)
        axes[1][i].axis('off')

print(f'Total {len(mask)} images')
plt.show()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
df

# Generate mask image

In [ ]:
for c, row in df.iterrows():
    f =  row['filename']
    img = cv2.imread(PATH_IMG + f)
    # Background
    img_mask = np.zeros(img.shape) 
    # Mask pixels are labeled to (1,1,1), (2,2,2),...
    for i, label in enumerate(row['labels']):
        for j, region_name in enumerate(region_names):
            if label == region_names[j]:
                img_mask = cv2.drawContours(img_mask, [row['contours'][i]], -1, thickness=-1, color=(j+1,j+1,j+1))
    
    cv2.imwrite(PATH_MASK + f.split('.')[0] + '_mask.png', img_mask)
    print(f"Saved mask: {PATH_MASK + f.split('.')[0] + '_mask.png'}") # Format shall be 'png' in order not to be compressed.

In [ ]:
# Confirm mask labels
masks = glob(PATH_MASK + '*.png')
img_mask = cv2.imread(masks[0])
print('Mask labels:\n', np.unique(img_mask.reshape(-1, img_mask.shape[2]), axis=0))

In [ ]:
# Confirm mask images
sample_masks = masks[:5]
fig, axes = plt.subplots(1, 5, figsize=(20, 6))
for i, mask in enumerate(sample_masks):
    img_mask = cv2.imread(mask)
    axes[i].imshow(img_mask/np.max(img_mask))
    axes[i].axis('off')
    axes[i].set_title(mask.split('\\')[-1])